In [726]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection, preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier

In [5]:
df=pd.read_csv(r'C:\Users\aditya_jyoti\Documents\Docs\Hackathon\train_Df64byy.csv')

In [6]:
df1=df.copy()

In [7]:
df.isnull().sum()

ID                             0
City_Code                      0
Region_Code                    0
Accomodation_Type              0
Reco_Insurance_Type            0
Upper_Age                      0
Lower_Age                      0
Is_Spouse                      0
Health Indicator           11691
Holding_Policy_Duration    20251
Holding_Policy_Type        20251
Reco_Policy_Cat                0
Reco_Policy_Premium            0
Response                       0
dtype: int64

In [8]:
df.T.apply(lambda x: x.nunique(), axis=1)

ID                         50882
City_Code                     36
Region_Code                 5316
Accomodation_Type              2
Reco_Insurance_Type            2
Upper_Age                     58
Lower_Age                     60
Is_Spouse                      2
Health Indicator               9
Holding_Policy_Duration       15
Holding_Policy_Type            4
Reco_Policy_Cat               22
Reco_Policy_Premium         6977
Response                       2
dtype: int64

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50882 entries, 0 to 50881
Data columns (total 14 columns):
ID                         50882 non-null int64
City_Code                  50882 non-null object
Region_Code                50882 non-null int64
Accomodation_Type          50882 non-null object
Reco_Insurance_Type        50882 non-null object
Upper_Age                  50882 non-null int64
Lower_Age                  50882 non-null int64
Is_Spouse                  50882 non-null object
Health Indicator           39191 non-null object
Holding_Policy_Duration    30631 non-null object
Holding_Policy_Type        30631 non-null float64
Reco_Policy_Cat            50882 non-null int64
Reco_Policy_Premium        50882 non-null float64
Response                   50882 non-null int64
dtypes: float64(2), int64(6), object(6)
memory usage: 5.4+ MB


In [10]:
df['Holding_Policy_Duration']=df['Holding_Policy_Duration'].replace('14+',15)

In [11]:
df=df.astype({"Holding_Policy_Type":'category', "Reco_Policy_Cat":'category',"City_Code":'category',"Accomodation_Type":'category',"Reco_Insurance_Type":'category',"Holding_Policy_Duration":'category',"Is_Spouse":'category',"Health Indicator":'category','Region_Code':'category'})

In [12]:
#No missing values in num variables
num_vars=df.columns[df.dtypes!='category']
cat_vars=df.columns[df.dtypes=='category']

# MISSING VALUE ANALYSIS

In [13]:
df['Health Indicator']=df['Health Indicator'].cat.codes
df['Health Indicator']=df['Health Indicator'].replace(-1, np.nan)
# df1['Health Indicator'].isnull().sum()

In [14]:
df2=pd.get_dummies(df, columns=['City_Code','Accomodation_Type','Reco_Insurance_Type','Is_Spouse'])

In [29]:
from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer()
data_with_imputed_values = pd.DataFrame(my_imputer.fit_transform(df2))

In [30]:
data_with_imputed_values.columns=df2.columns
data_with_imputed_values.index=df2.index

# OUTLIER ANALYSIS

In [31]:
def remove_outlier(df_in, col_name):
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
    return df_out

In [32]:
data_with_imputed_values.columns

Index(['ID', 'Region_Code', 'Upper_Age', 'Lower_Age', 'Health Indicator',
       'Holding_Policy_Duration', 'Holding_Policy_Type', 'Reco_Policy_Cat',
       'Reco_Policy_Premium', 'Response', 'City_Code_C1', 'City_Code_C10',
       'City_Code_C11', 'City_Code_C12', 'City_Code_C13', 'City_Code_C14',
       'City_Code_C15', 'City_Code_C16', 'City_Code_C17', 'City_Code_C18',
       'City_Code_C19', 'City_Code_C2', 'City_Code_C20', 'City_Code_C21',
       'City_Code_C22', 'City_Code_C23', 'City_Code_C24', 'City_Code_C25',
       'City_Code_C26', 'City_Code_C27', 'City_Code_C28', 'City_Code_C29',
       'City_Code_C3', 'City_Code_C30', 'City_Code_C31', 'City_Code_C32',
       'City_Code_C33', 'City_Code_C34', 'City_Code_C35', 'City_Code_C36',
       'City_Code_C4', 'City_Code_C5', 'City_Code_C6', 'City_Code_C7',
       'City_Code_C8', 'City_Code_C9', 'Accomodation_Type_Owned',
       'Accomodation_Type_Rented', 'Reco_Insurance_Type_Individual',
       'Reco_Insurance_Type_Joint', 'Is_Spouse

In [24]:
df2.shape

(50882, 52)

In [33]:
data_with_imputed_values=remove_outlier(data_with_imputed_values,'Reco_Policy_Premium')

In [36]:
data_with_imputed_values.shape

(50061, 52)

# standardization

In [39]:
data_with_imputed_values[['Region_Code', 'Upper_Age', 'Lower_Age', 'Health Indicator','Holding_Policy_Duration', 'Holding_Policy_Type', 'Reco_Policy_Cat','Reco_Policy_Premium']]=StandardScaler().fit_transform(data_with_imputed_values[['Region_Code', 'Upper_Age', 'Lower_Age', 'Health Indicator','Holding_Policy_Duration', 'Holding_Policy_Type', 'Reco_Policy_Cat','Reco_Policy_Premium']])

In [40]:
data_with_imputed_values['Holding_Policy_Type'].value_counts()

 0.004568    20009
 0.710439    12997
-1.807057     8065
-0.548309     4947
 1.969186     4043
Name: Holding_Policy_Type, dtype: int64

In [41]:
data_with_imputed_values=data_with_imputed_values.astype({'Health Indicator':'category','Holding_Policy_Type':'category', 'Reco_Policy_Cat':'category','Holding_Policy_Duration':'category','City_Code_C1':'category', 'City_Code_C10':'category',
       'City_Code_C11':'category', 'City_Code_C12':'category', 'City_Code_C13':'category', 'City_Code_C14':'category',
       'City_Code_C15':'category', 'City_Code_C16':'category', 'City_Code_C17':'category', 'City_Code_C18':'category',
       'City_Code_C19':'category', 'City_Code_C2':'category', 'City_Code_C20':'category', 'City_Code_C21':'category',
       'City_Code_C22':'category', 'City_Code_C23':'category', 'City_Code_C24':'category', 'City_Code_C25':'category',
       'City_Code_C26':'category', 'City_Code_C27':'category', 'City_Code_C28':'category', 'City_Code_C29':'category',
       'City_Code_C3':'category', 'City_Code_C30':'category', 'City_Code_C31':'category', 'City_Code_C32':'category',
       'City_Code_C33':'category', 'City_Code_C34':'category', 'City_Code_C35':'category', 'City_Code_C36':'category',
       'City_Code_C4':'category', 'City_Code_C5':'category', 'City_Code_C6':'category', 'City_Code_C7':'category',
       'City_Code_C8':'category', 'City_Code_C9':'category', 'Accomodation_Type_Owned':'category',
       'Accomodation_Type_Rented':'category', 'Reco_Insurance_Type_Individual':'category',
       'Reco_Insurance_Type_Joint':'category', 'Is_Spouse_No':'category', 'Is_Spouse_Yes':'category','Region_Code':'category'})

In [42]:
data_with_imputed_values.columns[data_with_imputed_values.dtypes!='category']

Index(['ID', 'Upper_Age', 'Lower_Age', 'Reco_Policy_Premium', 'Response'], dtype='object')

In [45]:
data_with_imputed_values.head()

,ID,Region_Code,Upper_Age,Lower_Age,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium,Response,...,City_Code_C6,City_Code_C7,City_Code_C8,City_Code_C9,Accomodation_Type_Owned,Accomodation_Type_Rented,Reco_Insurance_Type_Individual,Reco_Insurance_Type_Joint,Is_Spouse_No,Is_Spouse_Yes
0,1.0,1.037601,-0.492694,-0.377296,-1.138848,2.413950,0.710439,1.087812,-0.364394,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
1,2.0,-0.433735,1.785492,-1.189619,-0.349516,0.006245,0.004568,1.087812,2.724060,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
2,3.0,1.401925,-0.726354,-0.609388,0.001495,-1.397765,-1.807057,0.615071,-1.047773,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,4.0,1.855399,0.441946,0.318980,-1.138848,2.413950,0.710439,0.615071,0.641865,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
4,5.0,0.319482,-0.025374,0.086888,-0.349516,-0.853234,-1.807057,0.142330,-0.564599,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0


# MODEL BUILDING

# GRADIENT BOOSTING

In [69]:
X=data_with_imputed_values.loc[:, data_with_imputed_values.columns != 'ID']
y=data_with_imputed_values['Response']
X.drop(columns=['Response'],inplace=True)

C:\Python\lib\site-packages\pandas\core\frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [145]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=100)

In [146]:
from imblearn.pipeline import make_pipeline
from imblearn.combine import SMOTEENN
smote = SMOTE(k_neighbors=5, n_jobs=-1,random_state=777)
smote_enn = make_pipeline(SimpleImputer(), SMOTEENN(smote=smote))
sm_xtrain, sm_ytrain = smote_enn.fit_resample(X, y)

In [ ]:
# from imblearn.pipeline import make_pipeline
# from imblearn.combine import SMOTEENN
# smote = SMOTE(k_neighbors=5, n_jobs=-1,random_state=777)
# smote_enn = make_pipeline(SimpleImputer(), smote=smote)
# sm_xtrain, sm_ytrain = smote_enn.fit_resample(X, y)

In [101]:
# from imblearn.over_sampling import BorderlineSMOTE, SMOTE, ADASYN, SMOTENC, RandomOverSampler
# sm = SMOTE(random_state=777)
# sm_xtrain, sm_ytrain = sm.fit_sample(X_train,y_train)

In [112]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(n_estimators=600)
gb.fit(sm_xtrain, sm_ytrain)
y_pred1 = gb.predict(X_test)
accuracy_gb = metrics.accuracy_score(test_y,y_pred1)
print(accuracy_gb)

0.6556476580445421


In [113]:
report = classification_report(test_y,y_pred1)
print(report)

              precision    recall  f1-score   support

           0       0.82      0.69      0.75      7575
           1       0.36      0.54      0.43      2438

    accuracy                           0.66     10013
   macro avg       0.59      0.62      0.59     10013
weighted avg       0.71      0.66      0.67     10013



# RANDOM FOREST

In [204]:
from sklearn.feature_selection import SelectFromModel

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rndmf = SelectFromModel(RandomForestClassifier(n_estimators=500,class_weight='balanced'))
rndmf.fit(sm_xtrain, sm_ytrain)

In [207]:
from sklearn.ensemble import RandomForestClassifier
rndmf = RandomForestClassifier(n_estimators=500,class_weight='balanced')
rndmf.fit(sm_xtrain, sm_ytrain)
y_pred1 = rndmf.predict(X_test)
accuracy_rndmf = metrics.accuracy_score(test_y,y_pred1)
print(accuracy_rndmf)

0.7131728752621592


In [215]:
X_train[['Region_Code', 'Upper_Age', 'Lower_Age', 'Health Indicator',
       'Holding_Policy_Duration', 'Holding_Policy_Type', 'Reco_Policy_Cat',
       'Reco_Policy_Premium']].head()

,Region_Code,Upper_Age,Lower_Age,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium
31459,-0.205594,-0.492694,-0.377296,-1.138848,-1.397765,0.710439,0.299910,-0.814527
14372,-1.006545,-0.609524,-0.551365,-0.349516,-0.036438,-1.807057,0.930232,0.698262
44710,0.139075,-0.609524,-0.493342,-1.138848,2.141685,0.710439,0.930232,-0.594040
12344,-0.289129,0.909266,1.015256,-1.138848,-1.125499,0.710439,1.087812,1.256219
632,-1.208012,-0.375864,-0.319273,-0.349516,0.780358,0.710439,0.930232,0.063888


In [206]:
selected_feat= X_train.columns[(rndmf.get_support())]
print(selected_feat)

Index(['Region_Code', 'Upper_Age', 'Lower_Age', 'Health Indicator',
       'Holding_Policy_Duration', 'Holding_Policy_Type', 'Reco_Policy_Cat',
       'Reco_Policy_Premium'],
      dtype='object')


In [ ]:
report = classification_report(test_y,y_pred1)
print(report)

In [208]:
from sklearn.metrics import roc_auc_score
roc_auc_score(test_y, y_pred1)

0.7542409105553706

In [209]:
#without class weigt balanced
report = classification_report(test_y,y_pred1)
print(report)

              precision    recall  f1-score   support

           0       0.93      0.67      0.78      7575
           1       0.45      0.83      0.59      2438

    accuracy                           0.71     10013
   macro avg       0.69      0.75      0.68     10013
weighted avg       0.81      0.71      0.73     10013



In [210]:
Conf_metrics_sm_rndm = metrics.confusion_matrix(test_y, y_pred1,labels=[0,1])
print(Conf_metrics_sm_rndm)

[[5107 2468]
 [ 404 2034]]


# CATBOOST

In [180]:
from catboost import CatBoostClassifier
CatBoost=CatBoostClassifier()
parameters = {'depth'         : [4,5,6,7,8,9, 10],
            'learning_rate' : [0.01,0.02,0.03,0.04],
            'iterations'    : [10, 20,30,40,50,60,70,80,90, 100]
             }

Grid_CBC = GridSearchCV(estimator=CatBoost, param_grid = parameters,
                               cv = 2,  n_jobs=-1)
Grid_CBC.fit(sm_xtrain,sm_ytrain)

# Results from Grid Search
print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n",Grid_CBC.best_estimator_)
print("\n The best score across ALL searched params:\n",Grid_CBC.best_score_)
print("\n The best parameters across ALL searched params:\n",Grid_CBC.best_params_)

0:	learn: 0.6865569	total: 223ms	remaining: 22s
1:	learn: 0.6808722	total: 445ms	remaining: 21.8s
2:	learn: 0.6739747	total: 785ms	remaining: 25.4s
3:	learn: 0.6687388	total: 1.18s	remaining: 28.2s
4:	learn: 0.6641737	total: 1.55s	remaining: 29.4s
5:	learn: 0.6569254	total: 1.82s	remaining: 28.6s
6:	learn: 0.6527635	total: 2.15s	remaining: 28.6s
7:	learn: 0.6451913	total: 2.44s	remaining: 28s
8:	learn: 0.6394364	total: 2.69s	remaining: 27.3s
9:	learn: 0.6353220	total: 3.06s	remaining: 27.6s
10:	learn: 0.6315548	total: 3.32s	remaining: 26.8s
11:	learn: 0.6279403	total: 3.59s	remaining: 26.3s
12:	learn: 0.6238487	total: 3.84s	remaining: 25.7s
13:	learn: 0.6194881	total: 4.13s	remaining: 25.4s
14:	learn: 0.6145244	total: 4.43s	remaining: 25.1s
15:	learn: 0.6099861	total: 4.74s	remaining: 24.9s
16:	learn: 0.6068495	total: 5.03s	remaining: 24.6s
17:	learn: 0.6007621	total: 5.36s	remaining: 24.4s
18:	learn: 0.5969145	total: 5.63s	remaining: 24s
19:	learn: 0.5942624	total: 5.89s	remaining: 23

In [194]:
from catboost import CatBoostClassifier
CatBoost=CatBoostClassifier()                                                              
parameters = {'depth':[10], 'iterations': [100], 'learning_rate': [0.02]}
Grid_CBC = GridSearchCV(estimator=CatBoost, param_grid = parameters,cv = 2,n_jobs=-1)
Grid_CBC.fit(sm_xtrain,sm_ytrain)

0:	learn: 0.6865569	total: 162ms	remaining: 16.1s
1:	learn: 0.6808722	total: 325ms	remaining: 15.9s
2:	learn: 0.6739747	total: 510ms	remaining: 16.5s
3:	learn: 0.6687388	total: 684ms	remaining: 16.4s
4:	learn: 0.6641737	total: 871ms	remaining: 16.5s
5:	learn: 0.6569254	total: 1.06s	remaining: 16.6s
6:	learn: 0.6527635	total: 1.25s	remaining: 16.6s
7:	learn: 0.6451913	total: 1.45s	remaining: 16.6s
8:	learn: 0.6394364	total: 1.65s	remaining: 16.7s
9:	learn: 0.6353220	total: 1.84s	remaining: 16.5s
10:	learn: 0.6315548	total: 2.02s	remaining: 16.4s
11:	learn: 0.6279403	total: 2.21s	remaining: 16.2s
12:	learn: 0.6238487	total: 2.39s	remaining: 16s
13:	learn: 0.6194881	total: 2.57s	remaining: 15.8s
14:	learn: 0.6145244	total: 2.73s	remaining: 15.5s
15:	learn: 0.6099861	total: 2.92s	remaining: 15.4s
16:	learn: 0.6068495	total: 3.09s	remaining: 15.1s
17:	learn: 0.6007621	total: 3.27s	remaining: 14.9s
18:	learn: 0.5969145	total: 3.46s	remaining: 14.7s
19:	learn: 0.5942624	total: 3.66s	remaining

GridSearchCV(cv=2, error_score=nan,
             estimator=<catboost.core.CatBoostClassifier object at 0x000001CB83AECDC8>,
             iid='deprecated', n_jobs=-1,
             param_grid={'depth': [10], 'iterations': [100],
                         'learning_rate': [0.02]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [199]:
y_pred4=Grid_CBC.predict(np.array(X_test))

In [200]:
from sklearn.metrics import roc_auc_score
roc_auc_score(test_y, y_pred4)

0.5747487119507685

In [201]:
Conf_metrics_sm_cb = metrics.confusion_matrix(test_y, y_pred4,labels=[0,1])
print(Conf_metrics_sm_cb)

[[1642 5933]
 [ 164 2274]]


In [202]:
#without class weigt balanced
report = classification_report(test_y,y_pred4)
print(report)

              precision    recall  f1-score   support

           0       0.91      0.22      0.35      7575
           1       0.28      0.93      0.43      2438

    accuracy                           0.39     10013
   macro avg       0.59      0.57      0.39     10013
weighted avg       0.76      0.39      0.37     10013



# SVC

In [162]:
from sklearn.svm import SVC
svm_classifier = SVC(random_state=101)
svm_classifier.fit(np.array(X_train), train_y)
y_pred5 = svm_classifier.predict(np.array(X_test))
accuracy_svc = metrics.accuracy_score(test_y,y_pred5)
print(accuracy_svc)

0.7565165285129332


In [165]:
from sklearn.metrics import roc_auc_score
roc_auc_score(test_y, y_pred5)

0.5

In [166]:
Conf_metrics_sm_xgb = metrics.confusion_matrix(test_y, y_pred5,labels=[0,1])
print(Conf_metrics_sm_xgb)

[[7575    0]
 [2438    0]]


# DATA PREPARATION AGAIN

In [226]:
df.head(2)

,ID,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium,Response
0,1,C3,3213,1,0,36,36,No,0.0,15,3.0,22,11628.0,0
1,2,C5,1117,0,1,75,22,No,1.0,NaN,NaN,22,30510.0,0


In [219]:
df['Accomodation_Type']=df['Accomodation_Type'].cat.codes

In [225]:
df['Reco_Insurance_Type']=df['Reco_Insurance_Type'].cat.codes

In [227]:
df['Is_Spouse']=df['Is_Spouse'].cat.codes

In [228]:
df['City_Code']=df['City_Code'].cat.codes

In [229]:
df.head()

,ID,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium,Response
0,1,22,3213,1,0,36,36,0,0.0,15,3.0,22,11628.0,0
1,2,31,1117,0,1,75,22,0,1.0,NaN,NaN,22,30510.0,0
2,3,31,3732,0,0,32,32,0,NaN,1.0,1.0,19,7450.0,1
3,4,16,4378,0,1,52,48,0,0.0,15,3.0,19,17780.0,0
4,5,34,2190,1,0,44,44,0,1.0,3.0,1.0,16,10404.0,0


In [231]:
df3=df.copy()

In [235]:
df3.isnull().sum()

ID                             0
City_Code                      0
Region_Code                    0
Accomodation_Type              0
Reco_Insurance_Type            0
Upper_Age                      0
Lower_Age                      0
Is_Spouse                      0
Health Indicator           11691
Holding_Policy_Duration    20251
Holding_Policy_Type        20251
Reco_Policy_Cat                0
Reco_Policy_Premium            0
Response                       0
dtype: int64

In [234]:
from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer()
data_with_imputed_values = pd.DataFrame(my_imputer.fit_transform(df3))
data_with_imputed_values.columns=df3.columns
data_with_imputed_values.index=df3.index

In [236]:
data_with_imputed_values.head()

,ID,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium,Response
0,1.0,22.0,3213.0,1.0,0.0,36.0,36.0,0.0,0.000000,15.000000,3.000000,22.0,11628.0,0.0
1,2.0,31.0,1117.0,0.0,1.0,75.0,22.0,0.0,1.000000,6.156769,2.439228,22.0,30510.0,0.0
2,3.0,31.0,3732.0,0.0,0.0,32.0,32.0,0.0,1.444694,1.000000,1.000000,19.0,7450.0,1.0
3,4.0,16.0,4378.0,0.0,1.0,52.0,48.0,0.0,0.000000,15.000000,3.000000,19.0,17780.0,0.0
4,5.0,34.0,2190.0,1.0,0.0,44.0,44.0,0.0,1.000000,3.000000,1.000000,16.0,10404.0,0.0


In [241]:
data_with_imputed_values.dtypes

ID                          float64
City_Code                  category
Region_Code                category
Accomodation_Type          category
Reco_Insurance_Type        category
Upper_Age                   float64
Lower_Age                   float64
Is_Spouse                  category
Health Indicator           category
Holding_Policy_Duration    category
Holding_Policy_Type        category
Reco_Policy_Cat            category
Reco_Policy_Premium         float64
Response                   category
dtype: object

In [240]:
data_with_imputed_values=data_with_imputed_values.astype({"Holding_Policy_Type":'category', "Reco_Policy_Cat":'category',"City_Code":'category',"Accomodation_Type":'category',"Reco_Insurance_Type":'category',"Holding_Policy_Duration":'category',"Is_Spouse":'category',"Health Indicator":'category','Region_Code':'category','Response':'category'})

In [242]:
def remove_outlier(df_in, col_name):
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
    return df_out

In [247]:
data_with_imputed_values=remove_outlier(data_with_imputed_values,'Reco_Policy_Premium')

In [249]:
data_with_imputed_values[['City_Code','Region_Code', 'Upper_Age', 'Lower_Age', 'Health Indicator','Holding_Policy_Duration', 'Holding_Policy_Type', 'Reco_Policy_Cat','Reco_Policy_Premium']]=StandardScaler().fit_transform(data_with_imputed_values[['City_Code','Region_Code', 'Upper_Age', 'Lower_Age', 'Health Indicator','Holding_Policy_Duration', 'Holding_Policy_Type', 'Reco_Policy_Cat','Reco_Policy_Premium']])

C:\Python\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Python\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [565]:
data_with_imputed_values.head()

,ID,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium,Response
0,1.0,0.571138,1.037601,1.0,0.0,-0.492694,-0.377296,0.0,-1.138848,2.413950,0.710439,1.087812,-0.364394,0.0
1,2.0,1.312036,-0.433735,0.0,1.0,1.785492,-1.189619,0.0,-0.349516,0.006245,0.004568,1.087812,2.724060,0.0
2,3.0,1.312036,1.401925,0.0,0.0,-0.726354,-0.609388,0.0,0.001495,-1.397765,-1.807057,0.615071,-1.047773,1.0
3,4.0,0.077206,1.855399,0.0,1.0,0.441946,0.318980,0.0,-1.138848,2.413950,0.710439,0.615071,0.641865,0.0
4,5.0,1.559002,0.319482,1.0,0.0,-0.025374,0.086888,0.0,-0.349516,-0.853234,-1.807057,0.142330,-0.564599,0.0


In [600]:
X=data_with_imputed_values.loc[:, data_with_imputed_values.columns != 'ID']
y=data_with_imputed_values['Response']
X.drop(columns=['Response'],inplace=True)

C:\Python\lib\site-packages\pandas\core\frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [601]:
X=X[['City_Code', 'Region_Code', 'Upper_Age', 'Lower_Age','Health Indicator', 'Holding_Policy_Duration', 'Reco_Policy_Cat','Reco_Policy_Premium','Holding_Policy_Type','Reco_Insurance_Type','Is_Spouse']]

In [602]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.001, random_state=100)

In [599]:
X_train.columns

Index(['City_Code', 'Region_Code', 'Accomodation_Type', 'Reco_Insurance_Type',
       'Upper_Age', 'Lower_Age', 'Is_Spouse', 'Health Indicator',
       'Holding_Policy_Duration', 'Holding_Policy_Type', 'Reco_Policy_Cat',
       'Reco_Policy_Premium'],
      dtype='object')

In [603]:
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(y_train)
test_y = encoder.fit_transform(y_test)

In [604]:
from imblearn.pipeline import make_pipeline
from imblearn.combine import SMOTEENN
smote = SMOTE(k_neighbors=5, n_jobs=-1,random_state=777)
smote_enn = make_pipeline(SimpleImputer(), SMOTEENN(smote=smote))
sm_xtrain, sm_ytrain = smote_enn.fit_resample(X, y)

In [286]:
from sklearn.feature_selection import SelectFromModel

# RANDOM FOREST

In [287]:
from sklearn.ensemble import RandomForestClassifier
rndmf = SelectFromModel(RandomForestClassifier(n_estimators=500,class_weight='balanced'))
rndmf.fit(sm_xtrain, sm_ytrain)

SelectFromModel(estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                                 class_weight='balanced',
                                                 criterion='gini',
                                                 max_depth=None,
                                                 max_features='auto',
                                                 max_leaf_nodes=None,
                                                 max_samples=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=500, n_jobs=None,
                                                 oob_score=

In [288]:
selected_feat= X_train.columns[(rndmf.get_support())]
print(selected_feat)

Index(['City_Code', 'Region_Code', 'Reco_Policy_Cat', 'Reco_Policy_Premium'], dtype='object')


In [289]:
from sklearn.ensemble import RandomForestClassifier
rndmf = RandomForestClassifier(n_estimators=500,class_weight='balanced')
rndmf.fit(sm_xtrain, sm_ytrain)
y_pred1 = rndmf.predict(X_test)
accuracy_rndmf = metrics.accuracy_score(test_y,y_pred1)
print(accuracy_rndmf)

0.7545191251373214


In [674]:
X_train.columns

Index(['City_Code', 'Region_Code', 'Upper_Age', 'Lower_Age',
       'Health Indicator', 'Holding_Policy_Duration', 'Reco_Policy_Cat',
       'Reco_Policy_Premium', 'Holding_Policy_Type', 'Reco_Insurance_Type',
       'Is_Spouse'],
      dtype='object')

In [293]:
rndmf = RandomForestClassifier(n_estimators=500,class_weight='balanced')
n_estimators = [100, 300, 500, 800, 1200]
max_depth = [5, 8, 15, 25, 30]
min_samples_split = [2, 5, 10, 15, 100]
min_samples_leaf = [1, 2, 5, 10] 

hyperF = dict(n_estimators = n_estimators, max_depth = max_depth,  
              min_samples_split = min_samples_split, 
             min_samples_leaf = min_samples_leaf)

gridF = GridSearchCV(rndmf, hyperF, cv = 3, verbose = 1, 
                      n_jobs = -1)
bestF = gridF.fit(sm_xtrain, sm_ytrain)


Fitting 3 folds for each of 500 candidates, totalling 1500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  9.2min
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed: 24.5min
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed: 57.4min
[Parallel(n_jobs=-1)]: Done 1226 tasks      | elapsed: 117.7min
[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed: 157.6min finished


In [ ]:
# Results from Grid Search
print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n",bestF.best_estimator_)
print("\n The best score across ALL searched params:\n",bestF.best_score_)
print("\n The best parameters across ALL searched params:\n",bestF.best_params_)

In [605]:
RF_model= RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=30, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [588]:
X_train.shape

(50010, 12)

In [606]:
RF_model=RF_model.fit(sm_xtrain, sm_ytrain)

In [610]:
X_train.shape

(50010, 11)

In [294]:
# Results from Grid Search
print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n",bestF.best_estimator_)
print("\n The best score across ALL searched params:\n",bestF.best_score_)
print("\n The best parameters across ALL searched params:\n",bestF.best_params_)

 Results from Grid Search 

 The best estimator across ALL searched params:
 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=30, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

 The best score across ALL searched params:
 0.837041945489787

 The best parameters across ALL searched params:
 {'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 500}


In [663]:
len(X_train.columns)

11

In [607]:
y_pred1=RF_model.predict(X_test)

In [608]:
y_pred1_proba=RF_model.predict_proba(X_test)

In [673]:
from sklearn.metrics import roc_auc_score
roc_auc_score(test_y, y_pred1)

0.8299595141700407

In [576]:
from sklearn.metrics import roc_auc_score
roc_auc_score(test_y, y_pred1)

0.8299595141700407

In [544]:
from sklearn.metrics import roc_auc_score
roc_auc_score(test_y, y_pred1)

0.8168016194331984

In [ ]:
y

In [545]:
report = classification_report(test_y,y_pred1)
print(report)

              precision    recall  f1-score   support

           0       0.96      0.71      0.82        38
           1       0.52      0.92      0.67        13

    accuracy                           0.76        51
   macro avg       0.74      0.82      0.74        51
weighted avg       0.85      0.76      0.78        51



In [291]:
report = classification_report(test_y,y_pred1)
print(report)

              precision    recall  f1-score   support

           0       0.93      0.73      0.82      7575
           1       0.50      0.83      0.62      2438

    accuracy                           0.75     10013
   macro avg       0.71      0.78      0.72     10013
weighted avg       0.82      0.75      0.77     10013



In [636]:
Conf_metrics_sm_rndm = metrics.confusion_matrix(test_y, y_pred1,labels=[0,1])
print(Conf_metrics_sm_rndm)

[[28 10]
 [ 1 12]]


In [503]:
y_test.value_counts()

0.0    7575
1.0    2438
Name: Response, dtype: int64

# CATBOOST

In [371]:
# from catboost import CatBoostClassifier
# CatBoost=CatBoostClassifier() 
# CatBoost.fit(sm_xtrain, sm_ytrain)
# y_pred2 = CatBoost.predict(X_test)
# accuracy_rndmf = metrics.accuracy_score(test_y,y_pred1)
# print(accuracy_rndmf)

In [631]:
X_test.dtypes

City_Code                   float64
Region_Code                 float64
Upper_Age                   float64
Lower_Age                   float64
Health Indicator            float64
Holding_Policy_Duration     float64
Reco_Policy_Cat             float64
Reco_Policy_Premium         float64
Holding_Policy_Type         float64
Reco_Insurance_Type        category
Is_Spouse                  category
dtype: object

In [372]:
from catboost import CatBoostClassifier
CatBoost=CatBoostClassifier()
parameters = {'depth'         : [4,5,6,7,8,9, 10],
            'learning_rate' : [0.01,0.02,0.03,0.04],
            'iterations'    : [10, 20,30,40,50,60,70,80,90, 100,200]
             }

Grid_CBC = GridSearchCV(estimator=CatBoost, param_grid = parameters,
                               cv = 2,  n_jobs=-1)
Grid_CBC.fit(sm_xtrain,sm_ytrain)

# Results from Grid Search
print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n",Grid_CBC.best_estimator_)
print("\n The best score across ALL searched params:\n",Grid_CBC.best_score_)
print("\n The best parameters across ALL searched params:\n",Grid_CBC.best_params_)

0:	learn: 0.6900704	total: 94ms	remaining: 18.7s
1:	learn: 0.6856481	total: 187ms	remaining: 18.5s
2:	learn: 0.6829497	total: 291ms	remaining: 19.1s
3:	learn: 0.6803422	total: 391ms	remaining: 19.2s
4:	learn: 0.6768964	total: 489ms	remaining: 19.1s
5:	learn: 0.6724522	total: 613ms	remaining: 19.8s
6:	learn: 0.6693641	total: 745ms	remaining: 20.6s
7:	learn: 0.6670179	total: 865ms	remaining: 20.8s
8:	learn: 0.6643795	total: 967ms	remaining: 20.5s
9:	learn: 0.6609727	total: 1.07s	remaining: 20.3s
10:	learn: 0.6578679	total: 1.15s	remaining: 19.7s
11:	learn: 0.6558136	total: 1.23s	remaining: 19.2s
12:	learn: 0.6535211	total: 1.34s	remaining: 19.3s
13:	learn: 0.6508562	total: 1.47s	remaining: 19.5s
14:	learn: 0.6487476	total: 1.62s	remaining: 20s
15:	learn: 0.6465129	total: 1.79s	remaining: 20.6s
16:	learn: 0.6441299	total: 1.91s	remaining: 20.5s
17:	learn: 0.6409680	total: 2.02s	remaining: 20.4s
18:	learn: 0.6390923	total: 2.12s	remaining: 20.2s
19:	learn: 0.6368433	total: 2.21s	remaining:

162:	learn: 0.4713270	total: 19.2s	remaining: 4.35s
163:	learn: 0.4704265	total: 19.3s	remaining: 4.24s
164:	learn: 0.4698468	total: 19.4s	remaining: 4.12s
165:	learn: 0.4683459	total: 19.5s	remaining: 4s
166:	learn: 0.4677760	total: 19.7s	remaining: 3.89s
167:	learn: 0.4673303	total: 19.8s	remaining: 3.78s
168:	learn: 0.4667771	total: 20s	remaining: 3.67s
169:	learn: 0.4660470	total: 20.2s	remaining: 3.56s
170:	learn: 0.4654438	total: 20.3s	remaining: 3.44s
171:	learn: 0.4649362	total: 20.5s	remaining: 3.33s
172:	learn: 0.4644344	total: 20.6s	remaining: 3.21s
173:	learn: 0.4640110	total: 20.7s	remaining: 3.09s
174:	learn: 0.4631463	total: 20.7s	remaining: 2.96s
175:	learn: 0.4628863	total: 20.8s	remaining: 2.84s
176:	learn: 0.4622682	total: 20.9s	remaining: 2.72s
177:	learn: 0.4618930	total: 21s	remaining: 2.6s
178:	learn: 0.4614484	total: 21.1s	remaining: 2.48s
179:	learn: 0.4608714	total: 21.2s	remaining: 2.36s
180:	learn: 0.4605600	total: 21.3s	remaining: 2.24s
181:	learn: 0.459506

In [628]:
from catboost import CatBoostClassifier
CatBoost=CatBoostClassifier()
                                                 
                                                
                                                                                                           
CatBoost.fit(sm_xtrain,sm_ytrain)

Learning rate set to 0.051107
0:	learn: 0.6811986	total: 37.2ms	remaining: 37.2s
1:	learn: 0.6708061	total: 61.7ms	remaining: 30.8s
2:	learn: 0.6605016	total: 85.4ms	remaining: 28.4s
3:	learn: 0.6425298	total: 105ms	remaining: 26s
4:	learn: 0.6297798	total: 121ms	remaining: 24.1s
5:	learn: 0.6229510	total: 138ms	remaining: 22.9s
6:	learn: 0.6164408	total: 155ms	remaining: 22s
7:	learn: 0.6079225	total: 170ms	remaining: 21.1s
8:	learn: 0.6027527	total: 189ms	remaining: 20.8s
9:	learn: 0.5972508	total: 208ms	remaining: 20.6s
10:	learn: 0.5878215	total: 227ms	remaining: 20.4s
11:	learn: 0.5836162	total: 245ms	remaining: 20.2s
12:	learn: 0.5773595	total: 265ms	remaining: 20.1s
13:	learn: 0.5726358	total: 282ms	remaining: 19.9s
14:	learn: 0.5679727	total: 302ms	remaining: 19.8s
15:	learn: 0.5618494	total: 321ms	remaining: 19.7s
16:	learn: 0.5584690	total: 344ms	remaining: 19.9s
17:	learn: 0.5563627	total: 365ms	remaining: 19.9s
18:	learn: 0.5503875	total: 384ms	remaining: 19.8s
19:	learn: 0

164:	learn: 0.3718124	total: 3.78s	remaining: 19.1s
165:	learn: 0.3715032	total: 3.81s	remaining: 19.1s
166:	learn: 0.3712820	total: 3.83s	remaining: 19.1s
167:	learn: 0.3711654	total: 3.85s	remaining: 19.1s
168:	learn: 0.3710169	total: 3.88s	remaining: 19.1s
169:	learn: 0.3706267	total: 3.9s	remaining: 19s
170:	learn: 0.3702664	total: 3.92s	remaining: 19s
171:	learn: 0.3693602	total: 3.96s	remaining: 19s
172:	learn: 0.3691588	total: 3.98s	remaining: 19s
173:	learn: 0.3689806	total: 3.99s	remaining: 19s
174:	learn: 0.3680014	total: 4.01s	remaining: 18.9s
175:	learn: 0.3677843	total: 4.03s	remaining: 18.9s
176:	learn: 0.3674889	total: 4.05s	remaining: 18.8s
177:	learn: 0.3672655	total: 4.09s	remaining: 18.9s
178:	learn: 0.3666178	total: 4.12s	remaining: 18.9s
179:	learn: 0.3659490	total: 4.15s	remaining: 18.9s
180:	learn: 0.3657517	total: 4.2s	remaining: 19s
181:	learn: 0.3655913	total: 4.22s	remaining: 19s
182:	learn: 0.3652190	total: 4.24s	remaining: 18.9s
183:	learn: 0.3649684	total:

328:	learn: 0.3192136	total: 7.54s	remaining: 15.4s
329:	learn: 0.3188308	total: 7.56s	remaining: 15.3s
330:	learn: 0.3184850	total: 7.59s	remaining: 15.3s
331:	learn: 0.3182439	total: 7.61s	remaining: 15.3s
332:	learn: 0.3180031	total: 7.63s	remaining: 15.3s
333:	learn: 0.3178432	total: 7.67s	remaining: 15.3s
334:	learn: 0.3176930	total: 7.68s	remaining: 15.3s
335:	learn: 0.3175548	total: 7.71s	remaining: 15.2s
336:	learn: 0.3174097	total: 7.72s	remaining: 15.2s
337:	learn: 0.3172914	total: 7.74s	remaining: 15.2s
338:	learn: 0.3171348	total: 7.76s	remaining: 15.1s
339:	learn: 0.3168324	total: 7.78s	remaining: 15.1s
340:	learn: 0.3167173	total: 7.81s	remaining: 15.1s
341:	learn: 0.3165925	total: 7.83s	remaining: 15.1s
342:	learn: 0.3164547	total: 7.85s	remaining: 15s
343:	learn: 0.3163324	total: 7.87s	remaining: 15s
344:	learn: 0.3161824	total: 7.88s	remaining: 15s
345:	learn: 0.3160259	total: 7.9s	remaining: 14.9s
346:	learn: 0.3158468	total: 7.92s	remaining: 14.9s
347:	learn: 0.31569

495:	learn: 0.2952647	total: 11s	remaining: 11.2s
496:	learn: 0.2951549	total: 11.1s	remaining: 11.2s
497:	learn: 0.2950667	total: 11.1s	remaining: 11.2s
498:	learn: 0.2949517	total: 11.1s	remaining: 11.1s
499:	learn: 0.2948650	total: 11.1s	remaining: 11.1s
500:	learn: 0.2947491	total: 11.1s	remaining: 11.1s
501:	learn: 0.2946099	total: 11.2s	remaining: 11.1s
502:	learn: 0.2945240	total: 11.2s	remaining: 11s
503:	learn: 0.2944291	total: 11.2s	remaining: 11s
504:	learn: 0.2941449	total: 11.2s	remaining: 11s
505:	learn: 0.2940483	total: 11.3s	remaining: 11s
506:	learn: 0.2938949	total: 11.3s	remaining: 11s
507:	learn: 0.2938011	total: 11.3s	remaining: 11s
508:	learn: 0.2936122	total: 11.3s	remaining: 10.9s
509:	learn: 0.2934316	total: 11.4s	remaining: 10.9s
510:	learn: 0.2933476	total: 11.4s	remaining: 10.9s
511:	learn: 0.2932612	total: 11.4s	remaining: 10.9s
512:	learn: 0.2931715	total: 11.4s	remaining: 10.9s
513:	learn: 0.2930071	total: 11.5s	remaining: 10.8s
514:	learn: 0.2928849	tota

654:	learn: 0.2789392	total: 14.3s	remaining: 7.54s
655:	learn: 0.2788370	total: 14.3s	remaining: 7.52s
656:	learn: 0.2787724	total: 14.4s	remaining: 7.5s
657:	learn: 0.2786607	total: 14.4s	remaining: 7.47s
658:	learn: 0.2785317	total: 14.4s	remaining: 7.45s
659:	learn: 0.2784383	total: 14.4s	remaining: 7.43s
660:	learn: 0.2783437	total: 14.4s	remaining: 7.4s
661:	learn: 0.2782118	total: 14.5s	remaining: 7.38s
662:	learn: 0.2781567	total: 14.5s	remaining: 7.36s
663:	learn: 0.2780926	total: 14.5s	remaining: 7.33s
664:	learn: 0.2779872	total: 14.5s	remaining: 7.31s
665:	learn: 0.2779206	total: 14.5s	remaining: 7.29s
666:	learn: 0.2778452	total: 14.6s	remaining: 7.26s
667:	learn: 0.2777614	total: 14.6s	remaining: 7.24s
668:	learn: 0.2776570	total: 14.6s	remaining: 7.22s
669:	learn: 0.2775934	total: 14.6s	remaining: 7.2s
670:	learn: 0.2774182	total: 14.6s	remaining: 7.17s
671:	learn: 0.2773053	total: 14.6s	remaining: 7.15s
672:	learn: 0.2771775	total: 14.7s	remaining: 7.12s
673:	learn: 0.2

817:	learn: 0.2651737	total: 17.6s	remaining: 3.92s
818:	learn: 0.2651052	total: 17.6s	remaining: 3.9s
819:	learn: 0.2650313	total: 17.7s	remaining: 3.88s
820:	learn: 0.2649465	total: 17.7s	remaining: 3.85s
821:	learn: 0.2648770	total: 17.7s	remaining: 3.83s
822:	learn: 0.2648140	total: 17.7s	remaining: 3.81s
823:	learn: 0.2647337	total: 17.7s	remaining: 3.79s
824:	learn: 0.2646532	total: 17.8s	remaining: 3.77s
825:	learn: 0.2645908	total: 17.8s	remaining: 3.74s
826:	learn: 0.2645413	total: 17.8s	remaining: 3.72s
827:	learn: 0.2644541	total: 17.8s	remaining: 3.7s
828:	learn: 0.2644087	total: 17.8s	remaining: 3.67s
829:	learn: 0.2642920	total: 17.8s	remaining: 3.65s
830:	learn: 0.2642161	total: 17.9s	remaining: 3.63s
831:	learn: 0.2641324	total: 17.9s	remaining: 3.61s
832:	learn: 0.2640794	total: 17.9s	remaining: 3.59s
833:	learn: 0.2640081	total: 17.9s	remaining: 3.57s
834:	learn: 0.2639433	total: 17.9s	remaining: 3.54s
835:	learn: 0.2638695	total: 18s	remaining: 3.52s
836:	learn: 0.26

983:	learn: 0.2536931	total: 21s	remaining: 341ms
984:	learn: 0.2536201	total: 21s	remaining: 320ms
985:	learn: 0.2535517	total: 21s	remaining: 298ms
986:	learn: 0.2534710	total: 21s	remaining: 277ms
987:	learn: 0.2534253	total: 21s	remaining: 256ms
988:	learn: 0.2533707	total: 21.1s	remaining: 234ms
989:	learn: 0.2533192	total: 21.1s	remaining: 213ms
990:	learn: 0.2532543	total: 21.1s	remaining: 192ms
991:	learn: 0.2531980	total: 21.1s	remaining: 170ms
992:	learn: 0.2531421	total: 21.2s	remaining: 149ms
993:	learn: 0.2530962	total: 21.2s	remaining: 128ms
994:	learn: 0.2529918	total: 21.2s	remaining: 107ms
995:	learn: 0.2529453	total: 21.2s	remaining: 85.2ms
996:	learn: 0.2528813	total: 21.2s	remaining: 63.9ms
997:	learn: 0.2527885	total: 21.3s	remaining: 42.6ms
998:	learn: 0.2527193	total: 21.3s	remaining: 21.3ms
999:	learn: 0.2526280	total: 21.3s	remaining: 0us


In [632]:
y_pred2=CatBoost.predict(np.array(X_test))

In [633]:
roc_auc_score(test_y, y_pred2)

0.5981781376518218

In [634]:
report = classification_report(test_y,y_pred2)
print(report)

              precision    recall  f1-score   support

           0       0.81      0.66      0.72        38
           1       0.35      0.54      0.42        13

    accuracy                           0.63        51
   macro avg       0.58      0.60      0.57        51
weighted avg       0.69      0.63      0.65        51



In [442]:
report = classification_report(test_y,y_pred2)
print(report)

              precision    recall  f1-score   support

           0       0.91      0.60      0.72      7575
           1       0.39      0.81      0.53      2438

    accuracy                           0.65     10013
   macro avg       0.65      0.70      0.62     10013
weighted avg       0.78      0.65      0.67     10013



In [635]:
Conf_metrics_sm_cat = metrics.confusion_matrix(test_y, y_pred2,labels=[0,1])
print(Conf_metrics_sm_cat)

[[25 13]
 [ 6  7]]


# GRADIENT DESCENT

In [641]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(n_estimators=600)
gb.fit(np.array(sm_xtrain), sm_ytrain)
y_pred3 = gb.predict(X_test)

In [642]:
accuracy_gb = metrics.accuracy_score(test_y,y_pred3)
print(accuracy_gb)

0.6470588235294118


In [643]:
roc_auc_score(test_y, y_pred3)

0.5607287449392713

In [644]:
report = classification_report(test_y,y_pred3)
print(report)

              precision    recall  f1-score   support

           0       0.78      0.74      0.76        38
           1       0.33      0.38      0.36        13

    accuracy                           0.65        51
   macro avg       0.56      0.56      0.56        51
weighted avg       0.66      0.65      0.65        51



In [645]:
Conf_metrics_sm_gd = metrics.confusion_matrix(test_y, y_pred3,labels=[0,1])
print(Conf_metrics_sm_gd)

[[28 10]
 [ 8  5]]


# XGBOOST

In [320]:
xgb=XGBClassifier()
xgb.fit(np.array(sm_xtrain), sm_ytrain)
y_pred4 = xgb.predict(np.array(X_test))
accuracy_xgb = metrics.accuracy_score(test_y,y_pred4)
print(accuracy_xgb)

[22:58:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.7178667731948467


In [321]:
roc_auc_score(test_y, y_pred4)

0.6665242840937088

In [323]:
report = classification_report(test_y,y_pred4)
print(report)

              precision    recall  f1-score   support

           0       0.85      0.77      0.80      7575
           1       0.44      0.57      0.49      2438

    accuracy                           0.72     10013
   macro avg       0.64      0.67      0.65     10013
weighted avg       0.75      0.72      0.73     10013



In [325]:
Conf_metrics_sm_gd = metrics.confusion_matrix(test_y, y_pred4,labels=[0,1])
print(Conf_metrics_sm_gd)

[[5807 1768]
 [1057 1381]]


# LIGHT GBM

In [329]:
from lightgbm import LGBMClassifier
LGB=LGBMClassifier()
LGB.fit(np.array(sm_xtrain), sm_ytrain)
y_pred5 = LGB.predict(np.array(X_test))
accuracy_LGB = metrics.accuracy_score(test_y,y_pred5)
print(accuracy_LGB)

0.6678318186357735


In [330]:
roc_auc_score(test_y, y_pred5)

0.6305343069171561

In [332]:
Conf_metrics_sm_lgb = metrics.confusion_matrix(test_y, y_pred5,labels=[0,1])
print(Conf_metrics_sm_lgb)

[[5327 2248]
 [1078 1360]]


# DECISION TREE

In [715]:
from sklearn.tree import DecisionTreeClassifier
dt_cls = DecisionTreeClassifier(random_state=101)
dt_cls.fit(sm_xtrain,sm_ytrain)
y_pred6=dt_cls.predict(X_test)                                           

In [716]:
roc_auc_score(test_y, y_pred6)

0.8552631578947368

In [717]:
Conf_metrics_sm_dt = metrics.confusion_matrix(test_y, y_pred6,labels=[0,1])
print(Conf_metrics_sm_dt)

[[27 11]
 [ 0 13]]


In [690]:
tree_cv.predict(X_test)

array([0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1.,
       1., 0., 1., 0., 1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 0.,
       0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0.])

# TEST

In [613]:
Test=pd.read_csv(r'C:\Users\aditya_jyoti\Documents\Docs\Hackathon\test_YCcRUnU.csv')

In [614]:
Test.head()

,ID,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium
0,50883,C1,156,Owned,Individual,30,30,No,NaN,6.0,3.0,5,11934.0
1,50884,C4,7,Owned,Joint,69,68,Yes,X1,3.0,3.0,18,32204.8
2,50885,C1,564,Rented,Individual,28,28,No,X3,2.0,4.0,17,9240.0
3,50886,C3,1177,Rented,Individual,23,23,No,X3,3.0,3.0,18,9086.0
4,50887,C1,951,Owned,Individual,75,75,No,X3,NaN,NaN,5,22534.0


In [615]:
Test['Holding_Policy_Duration']=Test['Holding_Policy_Duration'].replace('14+',15)

In [616]:
Test=Test.astype({"Holding_Policy_Type":'category', "Reco_Policy_Cat":'category',"City_Code":'category',"Accomodation_Type":'category',"Reco_Insurance_Type":'category',"Holding_Policy_Duration":'category',"Is_Spouse":'category',"Health Indicator":'category','Region_Code':'category'})

In [617]:
Test['Accomodation_Type']=Test['Accomodation_Type'].cat.codes
Test['Reco_Insurance_Type']=Test['Reco_Insurance_Type'].cat.codes
Test['City_Code']=Test['City_Code'].cat.codes

In [618]:
Test['Health Indicator']=Test['Health Indicator'].cat.codes
Test['Health Indicator']=Test['Health Indicator'].replace(-1, np.nan)

In [619]:
Test.columns

Index(['ID', 'City_Code', 'Region_Code', 'Accomodation_Type',
       'Reco_Insurance_Type', 'Upper_Age', 'Lower_Age', 'Is_Spouse',
       'Health Indicator', 'Holding_Policy_Duration', 'Holding_Policy_Type',
       'Reco_Policy_Cat', 'Reco_Policy_Premium'],
      dtype='object')

In [620]:
Test=Test[['ID','City_Code', 'Region_Code', 'Upper_Age', 'Lower_Age','Health Indicator', 'Holding_Policy_Duration', 'Reco_Policy_Cat','Reco_Policy_Premium','Holding_Policy_Type','Accomodation_Type','Reco_Insurance_Type']]

In [621]:
from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer()
Test_data = pd.DataFrame(my_imputer.fit_transform(Test))
Test_data.columns=Test.columns
Test_data.index=Test.index

In [480]:
Test_data.columns

Index(['ID', 'City_Code', 'Region_Code', 'Upper_Age', 'Lower_Age',
       'Health Indicator', 'Holding_Policy_Duration', 'Reco_Policy_Cat',
       'Reco_Policy_Premium', 'Holding_Policy_Type'],
      dtype='object')

In [622]:
Test_data[['City_Code','Region_Code', 'Upper_Age', 'Lower_Age', 'Health Indicator','Holding_Policy_Duration', 'Holding_Policy_Type', 'Reco_Policy_Cat','Reco_Policy_Premium']]=StandardScaler().fit_transform(Test_data[['City_Code','Region_Code', 'Upper_Age', 'Lower_Age', 'Health Indicator','Holding_Policy_Duration', 'Holding_Policy_Type', 'Reco_Policy_Cat','Reco_Policy_Premium']])

In [623]:
Final_Test_data=Test_data.drop(columns=['ID'])

In [718]:
y_final=dt_cls.predict_proba(Final_Test_data)[:,1]

In [665]:
y_final

array([0.97152353, 0.678     , 0.98985772, ..., 0.052     , 0.90611931,
       0.772     ])

In [548]:
y_final

array([[0.04454356, 0.95545644],
       [0.27821699, 0.72178301],
       [0.004     , 0.996     ],
       ...,
       [0.97      , 0.03      ],
       [0.06355805, 0.93644195],
       [0.234     , 0.766     ]])

In [719]:
Test_data['Response']=y_final

In [720]:
Test_data.columns

Index(['ID', 'City_Code', 'Region_Code', 'Upper_Age', 'Lower_Age',
       'Health Indicator', 'Holding_Policy_Duration', 'Reco_Policy_Cat',
       'Reco_Policy_Premium', 'Holding_Policy_Type', 'Accomodation_Type',
       'Reco_Insurance_Type', 'Response'],
      dtype='object')

In [721]:
result=Test_data[['ID','Response']]

In [722]:
result['Response'].value_counts()

1.0    19408
0.0     2397
Name: Response, dtype: int64

In [627]:
result['Response'].value_counts()

1.0    19364
0.0     2441
Name: Response, dtype: int64

In [560]:
result['Response'].value_counts()

1.0    19224
0.0     2581
Name: Response, dtype: int64

In [515]:
result['Response'].value_counts()

1.0    20817
0.0      988
Name: Response, dtype: int64

In [508]:
result.head()

,ID,Response
0,50883.0,1.0
1,50884.0,1.0
2,50885.0,1.0
3,50886.0,1.0
4,50887.0,1.0


In [723]:
result.to_csv(r'C:\Users\aditya_jyoti\Documents\Docs\Hackathon\dt.csv')

In [656]:
check=pd.read_csv(r'C:\Users\aditya_jyoti\Documents\Docs\Hackathon\Random_forest.csv')

In [ ]:
predicted=[0 if i > 0.70 else 1 for i in y_prob1]

In [660]:
check['Response'].apply(lambda x:1 if x > 0.5 else 0 ).value_counts()

1    19224
0     2581
Name: Response, dtype: int64

In [561]:
X_train.columns

Index(['City_Code', 'Region_Code', 'Upper_Age', 'Lower_Age',
       'Health Indicator', 'Holding_Policy_Duration', 'Reco_Policy_Cat',
       'Reco_Policy_Premium', 'Holding_Policy_Type'],
      dtype='object')

In [562]:
df.columns

Index(['ID', 'City_Code', 'Region_Code', 'Accomodation_Type',
       'Reco_Insurance_Type', 'Upper_Age', 'Lower_Age', 'Is_Spouse',
       'Health Indicator', 'Holding_Policy_Duration', 'Holding_Policy_Type',
       'Reco_Policy_Cat', 'Reco_Policy_Premium', 'Response'],
      dtype='object')

In [563]:
df['Reco_Insurance_Type'].value_counts()

0    40536
1    10346
Name: Reco_Insurance_Type, dtype: int64

In [564]:
df['Is_Spouse'].value_counts()

0    42460
1     8422
Name: Is_Spouse, dtype: int64